<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark
!pip install pyspark
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import findspark
findspark.init()

In [3]:
# Start session
from pyspark import SparkContext, SparkConf
from pyspark.sql import  SparkSession

sc = SparkContext()

spark = SparkSession \
    .builder \
    .appName("My DE Capstone Project").getOrCreate()

24/11/06 21:17:11 WARN Utils: Your hostname, thangquang-Aspire-A715-41G resolves to a loopback address: 127.0.1.1; using 10.0.240.160 instead (on interface wlp4s0)
24/11/06 21:17:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/06 21:17:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Download The search term dataset from the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2024-11-06 21:17:15--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv.1’

searchterms.csv.1   100%[===================>] 227,99K   231KB/s    in 1,0s    

2024-11-06 21:17:17 (231 KB/s) - ‘searchterms.csv.1’ saved [233457/233457]



In [5]:
# Load the csv into a spark dataframe

In [6]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)

In [7]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [8]:
num_rows = df.count()
num_columns = len(df.columns)
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 10000
Number of columns: 4


In [9]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [10]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [11]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [12]:
df.printSchema()

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [13]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [14]:
df.filter(condition="searchterm == 'gaming laptop'").count()

499

In [15]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [16]:
from pyspark.sql.functions import desc

df.groupBy("searchterm").count().orderBy(desc("count")).show(5)

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [17]:
# The pretrained sales forecasting model is available at  the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

--2024-11-06 21:17:25--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 

169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1,5K) [application/x-tar]
Saving to: ‘model.tar.gz.1’

model.tar.gz.1      100%[===================>]   1,46K  --.-KB/s    in 0s      

2024-11-06 21:17:26 (844 MB/s) - ‘model.tar.gz.1’ saved [1490/1490]



In [18]:
!tar -xzf model.tar.gz

In [19]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [20]:
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load("sales_prediction.model")

In [21]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [22]:
from pyspark.ml.feature import VectorAssembler

def predict(year):
    # Assuming the features are 'day', 'month', and 'year'
    assembler = VectorAssembler(inputCols=['year'], outputCol='features')
    columns = ["year"]
    data = [[year]]
    df = spark.createDataFrame(data, columns)
    df = assembler.transform(df)
    predictions = model.transform(df)
    predictions.select('year', 'prediction').show()

predict(2023)

+----+------------------+
|year|        prediction|
+----+------------------+
|2023|175.16564294006457|
+----+------------------+

